# Concept GNN Pipeline – Colab GPU A100

Ce notebook prépare et exécute l'ensemble du pipeline (génération des données + entraînement GNN) sur un GPU A100. Chaque étape est isolée dans une cellule afin de pouvoir valider progressivement le processus avant les runs massifs.

## 1. Pré-requis
- Type d'instance : **GPU A100** (Runtime Colab → *Changer le type d'exécution* → GPU → A100).
- Dépôt GitHub organisé selon la structure `concept-gnn-pipeline`.
- Fichier `requirements.txt` à la racine du dépôt.

In [ ]:
!nvidia-smi

## 2. Installation des dépendances
- Cloner le dépôt
- Installer les dépendances (PyTorch GPU, torch_geometric, etc.)

In [ ]:
import os

# ⬇️ Renseigner l'URL du dépôt (public ou via token HTTPS)
REPO_URL = "https://github.com/<org>/concept-gnn-pipeline.git"
REPO_DIR = "/content/concept-gnn-pipeline"

if not os.path.exists(REPO_DIR):
    !git clone --depth 1 {REPO_URL} {REPO_DIR}
else:
    %cd {REPO_DIR}
    !git pull

%cd {REPO_DIR}

# Installation des dépendances
!pip install -U pip
!pip install -r requirements.txt

## 3. Vérification / édition des fichiers de configuration

Adapter les différents YAML/JSON si nécessaire avant de lancer les scripts.

In [ ]:
!ls config

# Exemple : afficher bins.yml
!cat config/bins.yml

## 4. Balayage métrique déterministe
Génère `analysis/effect_metric_sweep.csv` pour inspecter l’effet de chaque chaîne.

In [ ]:
!python scripts/sweep_metrics.py --config config/bins.yml --output analysis/effect_metric_sweep.csv
!head -n 5 analysis/effect_metric_sweep.csv

## 5. Génération de dataset avec couverture contrôlée
Paramètres définis dans `config/generator.yml`. Ajuster `--max-samples`, `--coverage-threshold`, `--min-per-bin` selon la taille voulue.

In [ ]:
DATASET_PATH = "analysis/train_dataset.jsonl"

!python scripts/generate_dataset.py \
    --config config/generator.yml \
    --max-samples 20000 \
    --coverage-threshold 0.7 \
    --min-per-bin 20 \
    --output {DATASET_PATH}

## 6. Inspection rapide du dataset
- Vérifier quelques enregistrements
- Visualiser la couverture des bins et la distribution des métriques

In [ ]:
import pandas as pd
import json

# Lecture partielle du JSONL
records = []
with open(DATASET_PATH) as f:
    for idx, line in enumerate(f):
        if idx >= 5000:  # limiter l'aperçu
            break
        rec = json.loads(line)
        metrics = rec["metrics"]
        metrics["sequence_id"] = rec["sequence_id"]
        records.append(metrics)

df = pd.DataFrame(records)
df.describe()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(6, 4))
sns.histplot(df['fft_anisotropy'], bins=30)
plt.title('Distribution de l\'anisotropie FFT')
plt.show()

plt.figure(figsize=(6, 4))
sns.histplot(df['mutual_info_local'], bins=30)
plt.title('Distribution de la MI locale')
plt.show()

## 7. Entraînement GNN (démarrage) 
Adapter les hyperparamètres dans `config/train.yml`. Cette cellule lance un training de base, sauvegarde les checkpoints, et écrit les métriques epoch par epoch.

In [ ]:
LOG_DIR = "analysis/run_colab"
!python scripts/train_gnn.py \
    --config config/train.yml \
    --dataset {DATASET_PATH} \
    --log-dir {LOG_DIR} \
    --epochs 20 \
    --checkpoint-every 5

## 8. Analyse des métriques d'entraînement

In [ ]:
import json
import matplotlib.pyplot as plt

records = []
with open(f"{LOG_DIR}/metrics_log.jsonl") as f:
    for line in f:
        records.append(json.loads(line))

epochs = [r['epoch'] for r in records]
train_loss = [r['train_loss'] for r in records]
train_acc = [r['train_acc'] for r in records]
val_acc = [r['val_acc'] for r in records]

fig, ax1 = plt.subplots(figsize=(7,4))
ax1.plot(epochs, train_loss, label='Train Loss', color='tab:red')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss', color='tab:red')

ax2 = ax1.twinx()
ax2.plot(epochs, train_acc, label='Train Acc', color='tab:blue')
ax2.plot(epochs, val_acc, label='Val Acc', color='tab:green')
ax2.set_ylabel('Accuracy')

fig.legend(loc='upper center', bbox_to_anchor=(0.5, 1.1), ncol=3)
plt.title('Suivi entraînement GNN')
plt.show()

## 9. Évaluation & Inversion
En utilisant le checkpoint de votre choix, lancez une évaluation complète ou un test d'inversion.

In [ ]:
!python scripts/evaluate_gnn.py \
    --checkpoint {LOG_DIR}/ckpt_epoch20.pt \
    --dataset analysis/test_dataset.jsonl \
    --topk 5

## 10. Sauvegarde / export des artifacts
Compresser logs + checkpoints, et les télécharger via Colab ou les envoyer sur Drive/Cloud.

In [ ]:
!tar -czf run_colab_artifacts.tar.gz {LOG_DIR} analysis/effect_metric_sweep.csv
from google.colab import files
files.download('run_colab_artifacts.tar.gz')

## 11. Étapes suivantes
- Ajuster la génération (augmenter `max-samples`, affiner les bins).
- Tester différentes architectures (Graph Transformer + pos enc, attention multi-échelle).
- Lancer l’entraînement en streaming Distribué (multi-GPU) sur cluster GH200/H100.
- Intégrer des pipelines d’inversion et d’estimation WebP dans un service temps réel.
